In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.8/782.8 kB 11.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [4]:
import zipfile
import os
import shutil
from ultralytics          import YOLO
from google.colab         import drive
import random
import tqdm

In [5]:
drive.mount('/content/drive/')

Mounted at /content/drive/


## YOLO_DATASET SHUFFLED + COCO

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/yolo_dataset.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/coco-seg_5000samples.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
img_dir = '/content/coco-images'
labels_dir = '/content/coco_annotation'
dst_dir = '/content/yolo_dataset'
def add_coco_dataset(img_dir, labels_dir, dst_dir, train_size=0.7, val_size=0.2, test_size=0.1):
  random.seed(42)
  imgs = os.listdir(img_dir)
  random.shuffle(imgs)
  length = len(imgs)

  train_end = int(train_size * length)
  val_end = train_end + int(val_size * length)

  train_imgs = imgs[:train_end]
  val_imgs = imgs[train_end:val_end]
  test_imgs = imgs[val_end:]

  for img in [img for img in train_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/train'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/train'))
  print(f'{train_end} images have been move.')
  for img in [img for img in val_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/val'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/val'))
  print(f'{val_end-train_end} images have been move.')
  for img in [img for img in test_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/test'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/test'))
  print(f'{length-val_end} images have been move.')

add_coco_dataset(img_dir, labels_dir, dst_dir)

3500 images have been move.
1000 images have been move.
500 images have been move.


In [ ]:
# Load a model
model = YOLO('yolov9c-seg.yaml')  # build a new model from YAML
model = YOLO('yolov9c-seg.pt')  # load a pretrained model (recommended for training)
model.info()

100%|██████████| 53.9M/53.9M [00:00<00:00, 63.3MB/s]

WARNING ⚠️ yolov9c-seg.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.0s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv9c-seg summary: 654 layers, 27897120 parameters, 0 gradients, 159.4 GFLOPs


(654, 27897120, 0, 159.41411839999998)

In [ ]:
# Train the model
model.train(data='/content/yolo_dataset/dataset.yaml', epochs=50, patience=10, imgsz=640, freeze=10, verbose=True, batch=-1, exist_ok=True,
      project='/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models', name='yolo_v9_coco/segment')

Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov9c-seg.pt, data=/content/yolo_dataset/dataset.yaml, epochs=50, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models, name=segment, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

train: Scanning /content/yolo_dataset/labels/train.cache... 50548 images, 38 backgrounds, 0 corrupt: 100%|██████████| 50548/50548 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/yolo_dataset/images/train/Otras_2_001471.jpg: 1 duplicate labels removed


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/yolo_dataset/labels/val.cache... 14434 images, 7 backgrounds, 0 corrupt: 100%|██████████| 14434/14434 [00:00<?, ?it/s]


...

      21/50      3.14G      1.011      1.451      0.661     0.9789         20        640: 100%|██████████| 16850/16850 [54:21<00:00,  5.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2406/2406 [04:50<00:00,  8.28it/s]


                   all      14434     103158      0.589      0.463      0.481      0.335        0.6      0.438      0.463      0.288

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      22/50      2.78G      1.007      1.446     0.6613     0.9792         33        640: 100%|██████████| 16850/16850 [54:04<00:00,  5.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2406/2406 [04:46<00:00,  8.40it/s]


                   all      14434     103158      0.605      0.454      0.484      0.337      0.611      0.438      0.466      0.289

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      23/50      2.75G      1.005      1.443     0.6552     0.9759         10        640:  84%|████████▍ | 14172/16850 [45:41<09:40,  4.61it/s]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_coco/segment'

In [ ]:
model = YOLO('/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_coco/segment/weights/best.pt')
model.info()

WARNING ⚠️ /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9/segment/weights/best.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.0s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv9c-seg summary: 654 layers, 27897891 parameters, 0 gradients, 159.4 GFLOPs


(654, 27897891, 0, 159.4184192)

In [ ]:
# Checking the test set (change from val rute to test rute in yaml file.)
model.val(data='/content/yolo_dataset/dataset.yaml')

In [ ]:
import shutil
shutil.move('/content/runs/segment/val/', '/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9/segment/test/')

'/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9/segment/test/'

In [ ]:
drive.flush_and_unmount()

## YOLO_DATASET VIDEO STREAM + COCO

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/video_yolo_dataset.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/coco-seg_5000samples.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
img_dir = '/content/coco-images'
labels_dir = '/content/coco_annotation'
dst_dir = '/content/yolo_dataset'
def add_coco_dataset(img_dir, labels_dir, dst_dir, train_size=0.8, val_size=0.2, test_size=1):
  imgs = os.listdir(img_dir)
  random.shuffle(imgs)
  length = len(imgs)

  train_end = int(train_size * length)
  val_end = train_end + int(val_size * length)

  train_imgs = imgs[:train_end]
  val_imgs = imgs[train_end:val_end]
  test_imgs = imgs[val_end:]

  for img in [img for img in train_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/train'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/train'))
  print(f'{train_end} images have been move.')
  for img in [img for img in val_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/val'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/val'))
  print(f'{val_end-train_end} images have been move.')
  for img in [img for img in test_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/test'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/test'))
  print(f'{length-val_end} images have been move.')

add_coco_dataset(img_dir, labels_dir, dst_dir)

4000 images have been move.
1000 images have been move.
0 images have been move.


In [ ]:
# Load a model
model = YOLO('yolov9c-seg.yaml')  # build a new model from YAML
model = YOLO('yolov9c-seg.pt')  # load a pretrained model (recommended for training)
model.info()

100%|██████████| 53.9M/53.9M [00:00<00:00, 73.7MB/s]

WARNING ⚠️ yolov9c-seg.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.7s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv9c-seg summary: 654 layers, 27897120 parameters, 0 gradients, 159.4 GFLOPs


(654, 27897120, 0, 159.41411839999998)

In [ ]:
# Train the model
model.train(data='/content/yolo_dataset/dataset.yaml', epochs=50, patience=10, imgsz=640, freeze=10, verbose=True, batch=-1, exist_ok=True,
      project='/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models', name='yolo_v9_video_coco/segment')

Ultralytics YOLOv8.2.22 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov9c-seg.pt, data=/content/yolo_dataset/dataset.yaml, epochs=50, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models, name=segment, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

100%|██████████| 755k/755k [00:00<00:00, 4.32MB/s]


Overriding model.yaml nc=80 with nc=81

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    212864  ultralytics.nn.modules.block.RepNCSPELAN4    [128, 256, 128, 64, 1]        
  3                  -1  1    164352  ultralytics.nn.modules.block.ADown           [256, 256]                    
  4                  -1  1    847616  ultralytics.nn.modules.block.RepNCSPELAN4    [256, 512, 256, 128, 1]       
  5                  -1  1    656384  ultralytics.nn.modules.block.ADown           [512, 512]                    
  6                  -1  1   2857472  ultralytics.nn.modules.block.RepNCSPELAN4    [512, 512, 512, 256, 1]       
  7                  -1  1    656384  ultralytic

100%|██████████| 6.23M/6.23M [00:00<00:00, 21.3MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=640
AutoBatch: CUDA:0 (Tesla T4) 14.75G total, 0.33G reserved, 0.25G allocated, 14.16G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


...

      28/50      6.01G     0.9858      1.355      0.586     0.9336        150        640: 100%|██████████| 4231/4231 [43:19<00:00,  1.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 446/446 [03:42<00:00,  2.01it/s]


                   all      12473     148031      0.599       0.44      0.478      0.341      0.595      0.431      0.463      0.293

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      29/50      5.98G     0.9825      1.345     0.5817     0.9313        127        640: 100%|██████████| 4231/4231 [43:18<00:00,  1.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 446/446 [03:43<00:00,  1.99it/s]


                   all      12473     148031      0.595      0.445      0.479      0.341      0.592      0.434      0.465      0.295

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      30/50      5.92G     0.9773      1.343     0.5785     0.9297        166        640:  91%|█████████ | 3836/4231 [39:21<04:02,  1.63it/s]

In [ ]:
model = YOLO('/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment/weights/last.pt')

WARNING ⚠️ /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment/weights/last.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.9s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



In [ ]:
model.train(resume=True)

Ultralytics YOLOv8.2.22 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment/weights/last.pt, data=/content/yolo_dataset/dataset.yaml, epochs=50, time=None, patience=10, batch=14, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models, name=segment, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream

100%|██████████| 755k/755k [00:00<00:00, 36.1MB/s]


TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment', view at http://localhost:6006/

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    212864  ultralytics.nn.modules.block.RepNCSPELAN4    [128, 256, 128, 64, 1]        
  3                  -1  1    164352  ultralytics.nn.modules.block.ADown           [256, 256]                    
  4                  -1  1    847616  ultralytics.nn.modules.block.RepNCSPELAN4    [256, 512, 256, 128, 1]       
  5                  -1  1    656384  ultralytics.nn.modules.block.ADown           [512, 512]                    
  6                  -1  1   2857472  ult

100%|██████████| 6.23M/6.23M [00:00<00:00, 195MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


...

      46/50      5.87G     0.9359      1.251     0.5164     0.9153         52        640: 100%|██████████| 4267/4267 [40:15<00:00,  1.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 446/446 [03:40<00:00,  2.03it/s]


                   all      12473     147680      0.678      0.546        0.6      0.442      0.691      0.525       0.58      0.377
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 36, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

16 epochs completed in 12.606 hours.
Optimizer stripped from /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment/weights/last.pt, 56.3MB
Optimizer stripped from /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment/weights/best.pt, 56.3MB

Validating /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment/weights/best.pt...
Ultralytics YOLOv8.2.22 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv9c-seg summary (fused): 411 layers, 27686979 parameters, 0 gradients, 158.0 GFLOPs


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:952: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv_transpose2d(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 446/446 [03:34<00:00,  2.08it/s]


                   all      12473     147680      0.695      0.558       0.61      0.453       0.68      0.545      0.588      0.387
                person      12473       6296      0.851      0.767      0.846      0.545      0.822      0.737      0.804      0.505
               bicycle      12473         56      0.652      0.468      0.514      0.325      0.598      0.429      0.434      0.228
                   car      12473        517      0.725      0.524       0.58       0.37      0.702      0.507      0.549      0.311
            motorcycle      12473         88      0.763      0.602      0.648      0.481      0.779      0.614      0.651      0.391
              airplane      12473         41      0.883      0.512       0.56      0.432      0.842      0.488      0.534      0.322
                   bus      12473         76      0.841      0.763      0.809      0.686      0.826      0.749      0.797      0.625
                 train      12473         40      0.864      0.791   

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       79, 80])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c6339596110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment'

<IPython.core.display.Javascript object>

In [ ]:
drive.flush_and_unmount()